# Workflow for obtaining the final CNS protein list applying some filters 

##### IRB Barcelona


The aim of this workflow is to obtain a list of CNS proteins which meet certain requirements such as being membrane proteins, targetable, with a PDB structure and being included in the MemProtMD repository. 

The simulations of these proteins will be run in future steps. 


### ______

### Table of contents for obtaining the worklflow

1. List CNS


    1.1. Bgee List
    1.2. Human Protein Atlas List
    1.3. Tissues List
    1.4. Overlap List
    


2. Filters

    a. Filter of biological interest

        2.1. Drug Targetable (Opentarget and Pharos) --> Targets for PD 
        2.2. Membrane protein (Uniprot)
      
     b. Filter of structure
        2.3. Length sequence > 100 aa (Uniprot)
        2.4. PDB structure (PDB)
       
     c. Filter for simulations and inserted in membrane
        2.5. In MemProtMD (MemProtMD)
        
    
    
    
3. Choose those PDBs that are not equal

### ______

#### Import libraries

In [1]:
import pandas as pd
import os
import json
from IPython.display import clear_output
import urllib.request
import json
from typing import Generator
import matplotlib.pyplot as plt
import numpy as np
import requests
import shutil
import json
import urllib.request
from typing import List

# DANI: No es necesario esto, y además no me funcionó instalarlo
#import wget

# DANI: En su lugar me hago mi propia función de wget.download
def download (url : str, directory_path : str):
    filename = (url.split('/')[-1]).split('?')[0]
    try:
        urllib.request.urlretrieve(url, directory_path + '/' + filename)
    except urllib.error.HTTPError as error:
        if error.code == 404:
            print('Missing file ' + url)
        else:
            raise Exception('Something went wrong with the url: ' + url)

#### Prepare folder where to store everything and downloaded databases

In [2]:
output_subdirectroy = 'results'
current_directory = os.getcwd()
working_directory = os.path.join(current_directory, output_subdirectroy)
    
folders = []
    
directory1 = 'Workflow_List_Proteins_List/2.Filters/2.1.OpenTargets'
directory2 = 'Workflow_List_Proteins_List/2.Filters/2.2.Membrane_UniProt'
directory3 = 'Workflow_List_Proteins_List/2.Filters/2.3.Length_Sequence'
directory4 = 'Workflow_List_Proteins_List/2.Filters/2.4.PDB'
directory5 = 'Workflow_List_Proteins_List/2.Filters/2.5.MemProtMD'
directory5A = 'Workflow_List_Proteins_List/2.Filters/2.5.MemProtMD/ToStudy_NonRepeatedSequences'
directory5B = 'Workflow_List_Proteins_List/2.Filters/2.5.MemProtMD/ToStudy_NonRepeatedSequences/unzipped'
filters = [directory1, directory2, directory3, directory4, directory5, directory5A, directory5B]

    
directory6 = 'Workflow_List_Proteins_List/1.Databases_CNS/1.1.Bgee_database'
directory7 = 'Workflow_List_Proteins_List/1.Databases_CNS/1.2.HPA_database'
directory8 = 'Workflow_List_Proteins_List/1.Databases_CNS/1.3.Tissues_database'
directory9 = 'Workflow_List_Proteins_List/1.Databases_CNS/1.4.Overlap'
databases = [directory6, directory7, directory8, directory9]

folders = filters + databases
    
path_dir = {}
    
for folder in folders:
    path = os.path.join(working_directory, folder)
    path_dir[folder] = path

In [3]:
# If the output subdirectory dues not exist then create it and download all databases
if not os.path.exists(working_directory):
    
    os.mkdir(working_directory)
    
    for folder in folders:   
        path = os.path.join(working_directory, folder)
        os.makedirs(path)
        path_dir[folder] = path
        
    
    # Download Bgee database 
    download('https://bgee.org/ftp/current/download/calls/expr_calls/Homo_sapiens_expr_simple.tsv.gz', path_dir[directory6])
   
    # Download Human Protein Atlas database
    download('https://www.proteinatlas.org/api/search_download.php?search=NOT%20tissue_category_rna%3Abrain%3Bnot%20detected&columns=g,gd,up,pe&compress=no&format=tsv', path_dir[directory7])
    os.rename(path_dir[directory7] + 'search_download.php', path_dir[directory7] + '/NOT.tsv')
    
    # Download Tissues database
    download('https://download.jensenlab.org/human_tissue_integrated_full.tsv', path_dir[directory8])
    
    # Download Open Targets databases
    os.chdir(path_dir[directory1])
    os.system('rsync -rpltvz --delete rsync.ebi.ac.uk::pub/databases/opentargets/platform/23.02/output/etl/json/associationByOverallDirect .')
    os.system('rsync -rpltvz --delete rsync.ebi.ac.uk::pub/databases/opentargets/platform/23.02/output/etl/json/targets .')
    
    # Download Pharos databases
    download('http://juniper.health.unm.edu/tcrd/download/PharosTCRD_UniProt_Mapping.tsv', path_dir[directory1])
    
    # Download UniProt database
    download('https://rest.uniprot.org/uniprotkb/stream?compressed=true&download=true&fields=accession%2Creviewed%2Cid%2Cprotein_name%2Cgene_names%2Corganism_name%2Clength%2Cft_intramem%2Cft_transmem&format=tsv&query=%28reviewed%3Atrue%29%20AND%20%28model_organism%3A9606%29%20AND%20%28existence%3A1%29%20AND%20%28reviewed%3Atrue%29', path_dir[directory2])
    

In [4]:
os.chdir(working_directory)

### 1. Selection of proteins belonging to the Central Nervous System

#### 1.1. Bgee List 

##### - Step 1. Visualize data and uniform it
We are going to visualize some of the list of the proteins in order to see that it was downloaded correctly.

In [5]:
# Specify the path where the file of all proteins is. Instead of unzipping the file in the folder, 
#we read it directly. 

import gzip
with gzip.open(path_dir[directory6]+'/Homo_sapiens_expr_simple.tsv.gz') as f:

    database_bgee= pd.read_csv(f,delimiter='\t')# Read the tsv. Tsv are limited by a t that's why it is called tsv.

database_bgee.head() # Visualization

,Gene ID,Gene name,Anatomical entity ID,Anatomical entity name,Expression,Call quality,FDR,Expression score,Expression rank
0,ENSG00000000003,TSPAN6,CL:0000015,male germ cell,present,gold quality,0.001825,99.04,449.0
1,ENSG00000000003,TSPAN6,CL:0000019,sperm,present,gold quality,0.001673,99.96,20.5
2,ENSG00000000003,TSPAN6,CL:0000023,oocyte,absent,gold quality,0.328065,62.47,17500.0
3,ENSG00000000003,TSPAN6,CL:0000083,epithelial cell of pancreas,present,gold quality,0.005262,83.30,7790.0
4,ENSG00000000003,TSPAN6,CL:0000094,granulocyte,present,silver quality,0.029202,46.45,25000.0


In [6]:
print('Number of entries of Bgee DB: ', len(database_bgee))

Number of entries of Bgee DB:  9093493


In [7]:
# Visualization of the unique values in the Anatomical entity name column: 

unique_anato_id_bgee_notone = database_bgee['Anatomical entity ID'].unique().tolist()
print(unique_anato_id_bgee_notone)

['CL:0000015', 'CL:0000019', 'CL:0000023', 'CL:0000083', 'CL:0000094', 'CL:0000115', 'CL:0000169', 'CL:0000576', 'CL:0000655', 'CL:0000738', 'CL:0000842', 'CL:0002079', 'CL:0002092', 'CL:0002255', 'CL:0002328', 'CL:0002336', 'UBERON:0000002', 'UBERON:0000006', 'UBERON:0000007', 'UBERON:0000014', 'UBERON:0000029', 'UBERON:0000043', 'UBERON:0000044', 'UBERON:0000057', 'UBERON:0000059', 'UBERON:0000074', 'UBERON:0000081', 'UBERON:0000082', 'UBERON:0000160', 'UBERON:0000167', 'UBERON:0000173', 'UBERON:0000178', 'UBERON:0000317', 'UBERON:0000331', 'UBERON:0000355', 'UBERON:0000362', 'UBERON:0000397', 'UBERON:0000399', 'UBERON:0000451', 'UBERON:0000458', 'UBERON:0000465', 'UBERON:0000467', 'UBERON:0000468', 'UBERON:0000473', 'UBERON:0000473 ∩ CL:0000089', 'UBERON:0000474', 'UBERON:0000922', 'UBERON:0000941', 'UBERON:0000945', 'UBERON:0000947', 'UBERON:0000948', 'UBERON:0000955', 'UBERON:0000956', 'UBERON:0000977', 'UBERON:0000979', 'UBERON:0000988', 'UBERON:0000989', 'UBERON:0000991 ∩ CL:000

From visualizing we have taken into account that the column Anatomical entity ID can contain more than one ID. We are going to filter this. We are going to stay with the first one and then add the second one when the list is created. 

In [8]:
CL_ids = [] # We store here the second ID

for id_ in unique_anato_id_bgee_notone: # For each value of the column Anatomical entity ID
    
    if '∩' in id_: # If the intersection symbol is found:
        splitted = id_.split() # The ID is splitted.
        CL_ids.append(splitted[2]) # The third value which starts by CL is stored in a list. 
        database_bgee.replace(to_replace=id_, value=splitted[0], inplace = True, regex=True) # Then, the value of the column where the ID is double is subsituted for its first term
        database_bgee[database_bgee['Anatomical entity ID'] == splitted[0]]

In [9]:
unique_anato_id_bgee = database_bgee['Anatomical entity ID'].unique().tolist()  # Creation of unique list.

unique_anato_id_bgee = unique_anato_id_bgee+CL_ids # Addition of the second ID to the list.

genes = database_bgee['Gene name'].unique().tolist() # Creation of the unique list again just in case. 

##### - Step 2. Find which Anatomical entity ID are from CNS. 

We do a filter of this file to see which proteins are from the Central Nervous System. 

First we go to https://www.ebi.ac.uk/ols/search?q=Central+Nervous+System&groupField=iri&start=0&ontology=uberon where the anatomical region we specified as Central Nervous System. We choose the first one " Central Nervous System" as it is what we want to filter. 

We see that the ID of the CNS is UBERON:0001017. As the list of proteins we have uses this types of ID in the column Anatomical entity ID; we want to filter this column and obtain only those genes that have this ID.

Moreover, theare are some genes that are classified as brain or other CNS parts and not as CNS. So we are also interested in collecting those. We want the Anatomical entity ID of CNS and other parts of CNS. To know these parts of CNS, in the uberon ontology of CNS there is a tree with the children of CNS. 

**What are we going to do**

Note:
We use better the ID rathen than the Anatomical entity name cause the ID is unique and the Anatomical entity name could be free text.


In [10]:

# Given a uberon code retunr a list with all its children uberon codes
def ols_get_code_children (uberon_code : str) -> List[str]:
    
    child_codes = []
    for page in range(0,3):# There are different pages because in one it does not fit all the data. In this case
                            # there are 3 pages containing data. So in the url we have to iterate for eaach page
                            # and from each one we extract the data. 
        # OLS url
    
        request_url = 'https://www.ebi.ac.uk/ols/api/ontologies/uberon/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252F'+uberon_code+'/hierarchicalDescendants?size=1000&page='+str(page)
        
        try:
            with urllib.request.urlopen(request_url) as response:
                parsed_response = json.loads(response.read().decode("utf-8"))
        # If the accession is not found then we can stop here
        except urllib.error.HTTPError as error:
            print('Error with request ' + request_url)
            raise ValueError('Something went wrong with the PDB request (error ' + str(error.code) + ')')
        # Mine children codes
        embedded = parsed_response.get('_embedded', None)
        if not embedded:
            return []
        children = embedded['terms']

        for child in children:
            #print(child['label'])
            child_code = child['iri'].split('/')[-1] #We select the last element of the iri because it is where the 
                                                    # UBERON code is.
            child_codes.append(child_code) # We append it to the list of children.
            
        # Return childrn codes
    return child_codes

all_descendents = ols_get_code_children('UBERON_0001017')

    
print(len(all_descendents))
print(all_descendents[0:3])

2339
['CL_0000117', 'CL_0000120', 'CL_0000626']


In [11]:
# For the following loop in the next cell (to search the web), the codes of the UBERON have to be with _ not :
# We change : for _

all_descendents_colon= [] #New list of values with _ not :

for i in range(0,len(all_descendents)): 
    all_descendents_colon.append(all_descendents[i].replace("_", ":" )) #We replace : for _ and we insert it to the list. 
    

In [12]:
# We select those codes from the table that are children of CNS (all descendents_colon)

PartsCNS_TF = database_bgee['Anatomical entity ID'].isin(all_descendents_colon)

PartsCNS=database_bgee[PartsCNS_TF]
PartsCNS.head()


,Gene ID,Gene name,Anatomical entity ID,Anatomical entity name,Expression,Call quality,FDR,Expression score,Expression rank
18,ENSG00000000003,TSPAN6,UBERON:0000007,pituitary gland,present,gold quality,1.000000e-14,93.38,3090.0
38,ENSG00000000003,TSPAN6,UBERON:0000451,prefrontal cortex,present,gold quality,4.110953e-14,74.56,11900.0
47,ENSG00000000003,TSPAN6,UBERON:0000941,cranial nerve II,present,gold quality,1.000000e-02,91.36,4030.0
51,ENSG00000000003,TSPAN6,UBERON:0000955,brain,present,gold quality,1.000000e-14,75.86,11300.0
52,ENSG00000000003,TSPAN6,UBERON:0000956,cerebral cortex,present,gold quality,1.000000e-14,74.74,11800.0


In [13]:
# We now select all the codes from the table that corresponds to the CNS: 

CNS = database_bgee[database_bgee['Anatomical entity ID']=='UBERON:0001017']

# We join the two tables obtained: the one for the CNS parts (children of CNS) and the one from CNS.
CNS_and_PartsCNS = pd.concat([CNS,PartsCNS])

CNS_and_PartsCNS.head()

,Gene ID,Gene name,Anatomical entity ID,Anatomical entity name,Expression,Call quality,FDR,Expression score,Expression rank
64,ENSG00000000003,TSPAN6,UBERON:0001017,central nervous system,present,gold quality,1.000000e-14,75.90,11200.0
611,ENSG00000000419,DPM1,UBERON:0001017,central nervous system,present,gold quality,1.000000e-14,90.04,4650.0
905,ENSG00000000457,SCYL3,UBERON:0001017,central nervous system,present,gold quality,1.310418e-08,69.63,14200.0
1189,ENSG00000000460,C1orf112,UBERON:0001017,central nervous system,present,gold quality,1.770695e-10,59.62,18800.0
1465,ENSG00000000938,FGR,UBERON:0001017,central nervous system,present,gold quality,1.000000e-14,68.54,14700.0


In [14]:
print('Number of entries when selecting CNS proteins and its parts: ',len(CNS_and_PartsCNS))

Number of entries when selecting CNS proteins and its parts:  1939872


##### - Step 3. Do a filter for Expression = present
We want only those genes that are expressed in order to have a protein list not a gene list.
We want to do it before the duplicates removal because if it is done after, the duplicates removal function drop_duplicates can select one gene that it is not expressed in some anatomical entity and in other one yes.  

In [15]:
# We select those which are expressed. 

expressed = CNS_and_PartsCNS[CNS_and_PartsCNS['Expression'] == 'present']

expressed.head()

,Gene ID,Gene name,Anatomical entity ID,Anatomical entity name,Expression,Call quality,FDR,Expression score,Expression rank
64,ENSG00000000003,TSPAN6,UBERON:0001017,central nervous system,present,gold quality,1.000000e-14,75.90,11200.0
611,ENSG00000000419,DPM1,UBERON:0001017,central nervous system,present,gold quality,1.000000e-14,90.04,4650.0
905,ENSG00000000457,SCYL3,UBERON:0001017,central nervous system,present,gold quality,1.310418e-08,69.63,14200.0
1189,ENSG00000000460,C1orf112,UBERON:0001017,central nervous system,present,gold quality,1.770695e-10,59.62,18800.0
1465,ENSG00000000938,FGR,UBERON:0001017,central nervous system,present,gold quality,1.000000e-14,68.54,14700.0


In [16]:
# We compare how much genes we have discarted, how much are not expressed. 

print('Number of entries when selecting CNS proteins and its parts: ',len(CNS_and_PartsCNS))
print('Number of entries when selecting CNS proteins and its parts with protein expression: ',len(expressed))

Number of entries when selecting CNS proteins and its parts:  1939872
Number of entries when selecting CNS proteins and its parts with protein expression:  1481642


##### - Step 4. Filter those repeated

We have obtained a list which includes proteins from the Central Nervous System and all its components. However, maybe there are some repeated. We have to remove the duplications in order to have unique values. 

We do not put any condition (prioritazing CNS e.g.) because we are only interested in those genes expressed, not other variables such as how much are expressed.

In [17]:
# We filter those genes that may be repeated in the table.

noduplicates= expressed.drop_duplicates(subset=['Gene ID'])
noduplicates.head()

,Gene ID,Gene name,Anatomical entity ID,Anatomical entity name,Expression,Call quality,FDR,Expression score,Expression rank
64,ENSG00000000003,TSPAN6,UBERON:0001017,central nervous system,present,gold quality,1.000000e-14,75.90,11200.0
611,ENSG00000000419,DPM1,UBERON:0001017,central nervous system,present,gold quality,1.000000e-14,90.04,4650.0
905,ENSG00000000457,SCYL3,UBERON:0001017,central nervous system,present,gold quality,1.310418e-08,69.63,14200.0
1189,ENSG00000000460,C1orf112,UBERON:0001017,central nervous system,present,gold quality,1.770695e-10,59.62,18800.0
1465,ENSG00000000938,FGR,UBERON:0001017,central nervous system,present,gold quality,1.000000e-14,68.54,14700.0


In [18]:
noduplicates[noduplicates['Gene name'] == 'SEMA3F']

,Gene ID,Gene name,Anatomical entity ID,Anatomical entity name,Expression,Call quality,FDR,Expression score,Expression rank
3958,ENSG00000001617,SEMA3F,UBERON:0001017,central nervous system,present,gold quality,1.714732e-13,74.33,12000.0


In [19]:
# We compare how much genes we have discarted:

print('Number of entries when selecting CNS proteins and its parts with protein expression: ',len(expressed))
print('Number of entries when selecting CNS proteins and its parts with protein expression and without duplicates genes: ', len(noduplicates))

Number of entries when selecting CNS proteins and its parts with protein expression:  1481642
Number of entries when selecting CNS proteins and its parts with protein expression and without duplicates genes:  55878


##### - Step 5. Uniprot overlap

When doing this overlap with uniprot we are making sure that: 

    1. Genes codify for proteins
    2. Genes are reviewed by Uniprot 
    
 
We choose the Gene ID and save them in the folder in the in order to introduce them in the Mapping of Uniprot with the objective to go from a ENSG id (Ensembl) to a Uniprot ID.
  

In [20]:
Gene_ENG_ID = noduplicates['Gene ID']

In [21]:
Gene_ENG_ID.to_csv(path_dir[directory6]+'/ID_Bgee_MapBy_ENSG_output.txt', header=None, index=None, sep=',', mode='w')

This list has to be loaded to UniProt Mapping tool, found in the following url: https://www.uniprot.org/id-mapping, with parameters: 
- From database: Ensembl
- To database: UniProtKB

Once obtained the mapping result, one should downloaded the resulting table.

In [22]:
# This cell may take time to run. 

import re
import time
import json
import zlib
from xml.etree import ElementTree
from urllib.parse import urlparse, parse_qs, urlencode
import requests
from requests.adapters import HTTPAdapter, Retry


POLLING_INTERVAL = 3
API_URL = "https://rest.uniprot.org"


retries = Retry(total=5, backoff_factor=0.25, status_forcelist=[500, 502, 503, 504])
session = requests.Session()
session.mount("https://", HTTPAdapter(max_retries=retries))


def check_response(response):
    try:
        response.raise_for_status()
    except requests.HTTPError:
        print(response.json())
        raise


def submit_id_mapping(from_db, to_db, ids):
    request = requests.post(
        f"{API_URL}/idmapping/run",
        data={"from": from_db, "to": to_db, "ids": ",".join(ids)},
    )
    check_response(request)
    return request.json()["jobId"]


def get_next_link(headers):
    re_next_link = re.compile(r'<(.+)>; rel="next"')
    if "Link" in headers:
        match = re_next_link.match(headers["Link"])
        if match:
            return match.group(1)


def check_id_mapping_results_ready(job_id):
    while True:
        request = session.get(f"{API_URL}/idmapping/status/{job_id}")
        check_response(request)
        j = request.json()
        if "jobStatus" in j:
            if j["jobStatus"] == "RUNNING":
                print(f"Retrying in {POLLING_INTERVAL}s")
                time.sleep(POLLING_INTERVAL)
            else:
                raise Exception(j["jobStatus"])
        else:
            return bool(j["results"] or j["failedIds"])


def get_batch(batch_response, file_format, compressed):
    batch_url = get_next_link(batch_response.headers)
    while batch_url:
        batch_response = session.get(batch_url)
        batch_response.raise_for_status()
        yield decode_results(batch_response, file_format, compressed)
        batch_url = get_next_link(batch_response.headers)


def combine_batches(all_results, batch_results, file_format):
    if file_format == "json":
        for key in ("results", "failedIds"):
            if key in batch_results and batch_results[key]:
                all_results[key] += batch_results[key]
    elif file_format == "tsv":
        return all_results + batch_results[1:]
    else:
        return all_results + batch_results
    return all_results


def get_id_mapping_results_link(job_id):
    url = f"{API_URL}/idmapping/details/{job_id}"
    request = session.get(url)
    check_response(request)
    return request.json()["redirectURL"]


def decode_results(response, file_format, compressed):
    if compressed:
        decompressed = zlib.decompress(response.content, 16 + zlib.MAX_WBITS)
        if file_format == "json":
            j = json.loads(decompressed.decode("utf-8"))
            return j
        elif file_format == "tsv":
            return [line for line in decompressed.decode("utf-8").split("\n") if line]
        elif file_format == "xlsx":
            return [decompressed]
        elif file_format == "xml":
            return [decompressed.decode("utf-8")]
        else:
            return decompressed.decode("utf-8")
    elif file_format == "json":
        return response.json()
    elif file_format == "tsv":
        return [line for line in response.text.split("\n") if line]
    elif file_format == "xlsx":
        return [response.content]
    elif file_format == "xml":
        return [response.text]
    return response.text


def get_xml_namespace(element):
    m = re.match(r"\{(.*)\}", element.tag)
    return m.groups()[0] if m else ""


def merge_xml_results(xml_results):
    merged_root = ElementTree.fromstring(xml_results[0])
    for result in xml_results[1:]:
        root = ElementTree.fromstring(result)
        for child in root.findall("{http://uniprot.org/uniprot}entry"):
            merged_root.insert(-1, child)
    ElementTree.register_namespace("", get_xml_namespace(merged_root[0]))
    return ElementTree.tostring(merged_root, encoding="utf-8", xml_declaration=True)


def print_progress_batches(batch_index, size, total):
    n_fetched = min((batch_index + 1) * size, total)
    print(f"Fetched: {n_fetched} / {total}")


def get_id_mapping_results_search(url):
    parsed = urlparse(url)
    query = parse_qs(parsed.query)
    file_format = query["format"][0] if "format" in query else "json"
    if "size" in query:
        size = int(query["size"][0])
    else:
        size = 500
        query["size"] = size
    compressed = (
        query["compressed"][0].lower() == "true" if "compressed" in query else False
    )
    parsed = parsed._replace(query=urlencode(query, doseq=True))
    url = parsed.geturl()
    request = session.get(url)
    check_response(request)
    results = decode_results(request, file_format, compressed)
    total = int(request.headers["x-total-results"])
    print_progress_batches(0, size, total)
    for i, batch in enumerate(get_batch(request, file_format, compressed), 1):
        results = combine_batches(results, batch, file_format)
        print_progress_batches(i, size, total)
    if file_format == "xml":
        return merge_xml_results(results)
    return results


def get_id_mapping_results_stream(url):
    if "/stream/" not in url:
        url = url.replace("/results/", "/results/stream/")
    request = session.get(url)
    check_response(request)
    parsed = urlparse(url)
    query = parse_qs(parsed.query)
    file_format = query["format"][0] if "format" in query else "json"
    compressed = (
        query["compressed"][0].lower() == "true" if "compressed" in query else False
    )
    return decode_results(request, file_format, compressed)



From database and to database value is choosen from https://rest.uniprot.org/configure/idmapping/fields

In [23]:
job_id = submit_id_mapping(from_db="Ensembl", to_db="UniProtKB", ids=list(Gene_ENG_ID))
if check_id_mapping_results_ready(job_id):
    link = get_id_mapping_results_link(job_id)
    results = get_id_mapping_results_search(link)
    # Equivalently using the stream endpoint which is more demanding
    # on the API and so is less stable:
    # results = get_id_mapping_results_stream(link)

Fetched: 500 / 78714
Fetched: 1000 / 78714
Fetched: 1500 / 78714
Fetched: 2000 / 78714
Fetched: 2500 / 78714
Fetched: 3000 / 78714
Fetched: 3500 / 78714
Fetched: 4000 / 78714
Fetched: 4500 / 78714
Fetched: 5000 / 78714
Fetched: 5500 / 78714
Fetched: 6000 / 78714
Fetched: 6500 / 78714
Fetched: 7000 / 78714
Fetched: 7500 / 78714
Fetched: 8000 / 78714
Fetched: 8500 / 78714
Fetched: 9000 / 78714
Fetched: 9500 / 78714
Fetched: 10000 / 78714
Fetched: 10500 / 78714
Fetched: 11000 / 78714
Fetched: 11500 / 78714
Fetched: 12000 / 78714
Fetched: 12500 / 78714
Fetched: 13000 / 78714
Fetched: 13500 / 78714
Fetched: 14000 / 78714
Fetched: 14500 / 78714
Fetched: 15000 / 78714
Fetched: 15500 / 78714
Fetched: 16000 / 78714
Fetched: 16500 / 78714
Fetched: 17000 / 78714
Fetched: 17500 / 78714
Fetched: 18000 / 78714
Fetched: 18500 / 78714
Fetched: 19000 / 78714
Fetched: 19500 / 78714
Fetched: 20000 / 78714
Fetched: 20500 / 78714
Fetched: 21000 / 78714
Fetched: 21500 / 78714
Fetched: 22000 / 78714
Fetched:

In [24]:
uniprot_bgee = []

for i in range(len(results['results'])):
    resulting = results['results'][i]['to']
    if resulting['entryType']== 'UniProtKB reviewed (Swiss-Prot)' and 'Homo sapiens' in str(resulting['organism']):
        uniprot_bgee.append(resulting['primaryAccession'])

In [25]:
len(uniprot_bgee)

18554

##### - Step 6. Correct list obtained from mapping

The resulting table from the UniProt mapping tool is loaded in the workflow. 

We select only those UniProt IDs that are reviewed and remove possible duplicates from the 'Entry' column.

In [26]:
Bgee_corrected = list(set(uniprot_bgee))

##### -  7. Save the CNS list from Bgee

In [27]:
with open(path_dir[directory6]+"/1.Bgee_ready_output.txt", 'w') as fp:
    for item in Bgee_corrected:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


# __________________________

#### 1.2. Human Protein Atlas

##### - Step 1. Downloading of HPA database
We have downloaded from Human Protein Atlas the proteins which should be in the human brain. 

Out of the 16465 genes detected above cut off in the human brain, 2685 genes have an elevated expression in the brain compared to other tissue types. So we have selected the list from the 16465 genes because although they are not hihgly expressed, they can have an important function.

Moreover, this database from the Human Protein Atlas is also considered from the Central Nervous System as it involves:

- Amygdala
- Basal ganglia
- Hippocampal formation
- Cerebral cortex
- Cerebellum
- Spinal cord
- Medulla obliongata
- Pons
- Midbrain
- Thalamus
- Hypothalamus
- White matter

In [29]:
# We load the dataset

database_hpa = pd.read_csv(path_dir[directory7]+'/NOT.tsv', delimiter='\t')

database_hpa.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/dbeltran/orozco55/desktop/model_cns_isabel/1.Obtain_CNSList_PrepareSimulations/results/Workflow_List_Proteins_List/1.Databases_CNS/1.2.HPA_database/NOT.tsv'

In [ ]:
print('Number of entries in the Human Protein Atlas DB: ', len(database_hpa))

In [ ]:
print('Values of unique values in Evidence column:  ', pd.unique(database_hpa['Evidence']))

##### - Step 2. Filter for Evidence at protein level

We are going to filter and stay with those that the Evidence column is 'present at protein level' in order to only have those genes that code for proteins and not those that code for non-coding RNA.

We do this filter in order to be more accurate with the list.


In [ ]:
database_hpa_protein = database_hpa[database_hpa['Evidence'] =='Evidence at protein level']

We see that some cells of the column 'Uniprot' are empty. This is because there is no Uniprot code for that gene. 
It was seen that 100 in the above table do not have a UniProt code.

In [ ]:
print('Number of unique UniProt IDs: ', len(pd.unique(database_hpa_protein['Uniprot'])))

##### - Step 3. Correct list 

In [ ]:
hpa_corrected = database_hpa_protein.drop_duplicates(subset=['Uniprot']) # There are duplicates from HPA

##### - 4. Save the CNS list from HPA

In [ ]:
hpa_corrected.to_csv(path_dir[directory7]+'/2.HPA_ready_output.csv', sep ='\t')

# __________________

#### 1.3 Tissues

##### - Step 1. Visualize data and uniform it

We are going to visualize some of the list of the proteins in order to see that it was downloaded correctly.

In [ ]:
database_tissues = pd.read_csv(path_dir[directory8]+'/human_tissue_integrated_full.tsv',delimiter='\t')

database_tissues.head()

In [ ]:
print('Number of entries in Tissues DB: ',len(database_tissues))

##### - Step 2. Find which Anatomical entity ID are from CNS. 

We do a filter of this file to see which proteins are from the Central Nervous System. 

First we go to https://www.ebi.ac.uk/ols/search?q=Central+Nervous+System&groupField=iri&start=0&ontology=uberon where the anatomical region we specified as Central Nervous System. We choose the first one " Central Nervous System" as it is what we want to filter. 

We see that the ID of the CNS is UBERON:0001017. As the list of proteins we have uses this types of ID in the column Anatomical entity ID; we want to filter this column and obtain only those genes that have this ID.

Moreover, theare are some genes that are classified as brain or other CNS parts and not as CNS. So we are also interested in collecting those. We want the Anatomical entity ID of CNS and other parts of CNS. To know these parts of CNS, in the uberon ontology of CNS there is a tree with the children of CNS. 

**What are we going to do**

Note:
We use better the ID rathen than the Anatomical entity name cause the ID is unique and the Anatomical entity name could be free text.


In [ ]:
import json
import urllib.request
from typing import List


# Given a uberon code retunr a list with all its children uberon codes
def ols_get_code_children (uberon_code : str) -> List[str]:
    # OLS url
    request_url = 'https://www.ebi.ac.uk/ols/api/ontologies/bto/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252F'+uberon_code+'/hierarchicalDescendants?size=1000'
    
    try:
        with urllib.request.urlopen(request_url) as response:
            parsed_response = json.loads(response.read().decode("utf-8"))
    # If the accession is not found then we can stop here
    except urllib.error.HTTPError as error:
        print('Error with request ' + request_url)
        raise ValueError('Something went wrong with the PDB request (error ' + str(error.code) + ')')
    # Mine children codes
    embedded = parsed_response.get('_embedded', None)
    if not embedded:
        return []
    children = embedded['terms']
    child_codes = []
    for child in children:
        #print(child['label'])
        child_code = child['iri'].split('/')[-1]
        child_codes.append(child_code)
    # Return childrn codes
    return child_codes

all_descendents = ols_get_code_children('BTO_0000227')
#all_descendents = ['GO_0043226']

    
print(len(all_descendents))
print(all_descendents[0:3])

In [ ]:
# For the following loop in the next cell (to search the web), the codes of the UBERON have to be with _ not :
# We change : for _

all_descendents_colon= [] #New list of values with _ not :

for i in range(0,len(all_descendents)): 
    all_descendents_colon.append(all_descendents[i].replace("_", ":" )) #We replace : for _ and we insert it to the list. 
    

In [ ]:
# We select those codes from the table that are children of CNS (all descendents_colon)

PartsCNS_TF = database_tissues['BTO:0000000'].isin(all_descendents_colon)

In [ ]:
PartsCNS=database_tissues[PartsCNS_TF]
PartsCNS

In [ ]:
# We now select all the codes from the table that corresponds to the CNS and Whole Body: 

Whole_Body = database_tissues[database_tissues['BTO:0000000']=='BTO:0001489']
CNS = Whole_Body[Whole_Body['BTO:0000000']=='BTO:0000227']

In [ ]:
# We join the two tables obtained: the one for the CNS parts (children of CNS) and the one from CNS.
CNS_and_PartsCNS = pd.concat([CNS,PartsCNS])
CNS_and_PartsCNS 

##### - Step 3. Filter those repeated.

We have obtained a list which includes proteins from the Central Nervous System and all its components. However, maybe there are some repeated. We have to remove the duplications in order to have 1. 

We do not put any condition (prioritazing CNS e.g.) because we are only interested in those genes expressed, not other variables such as how much are expressed.

In [ ]:
# We filter those genes that may be repeated in the table.

noduplicates= CNS_and_PartsCNS.drop_duplicates(subset=['18S_rRNA.1'])
noduplicates

In [ ]:
# We compare how much genes we have discarted:

len(CNS_and_PartsCNS),len(noduplicates)

##### - Step 4. Uniprot overlap

When doing this overlap with uniprot we are making sure that: 

    1. Genes codify for proteins
    2. Genes are reviewed by Uniprot 
    

We are going to map the Gene ID, which in principle it is from String; although we added also the Ensembl Protein ID to have wider results.

Ensembl IDs:

In [ ]:
Gene_ENS_ID = noduplicates['18S_rRNA']

String IDs: 

Here we add 9606 to the codes because for doing the mapping with Uniprot mapping tool, the String ID needs this number which identifies 'homo sapiens'. 

In [ ]:
Gene_String_ID = []

for i in Gene_ENS_ID: 
    String_ID = '9606.'+i
    Gene_String_ID.append(String_ID)
    

In [ ]:
del results 

In [ ]:

job_id = submit_id_mapping(
    from_db="Ensembl_Protein", to_db="UniProtKB", ids=list(Gene_ENS_ID)
)
if check_id_mapping_results_ready(job_id):
    link = get_id_mapping_results_link(job_id)
    results = get_id_mapping_results_search(link)
    # Equivalently using the stream endpoint which is more demanding
    # on the API and so is less stable:
    # results = get_id_mapping_results_stream(link)

In [ ]:
uniprot_hpa = []

for i in range(len(results['results'])):
    resulting = results['results'][i]['to']
    if resulting['entryType']== 'UniProtKB reviewed (Swiss-Prot)' and 'Homo sapiens' in str(resulting['organism']):
        uniprot_hpa.append(resulting['primaryAccession'])

In [ ]:
del results 

In [ ]:
job_id = submit_id_mapping(
    from_db="STRING", to_db="UniProtKB", ids=list(Gene_String_ID)
)
if check_id_mapping_results_ready(job_id):
    link = get_id_mapping_results_link(job_id)
    results = get_id_mapping_results_search(link)
    # Equivalently using the stream endpoint which is more demanding
    # on the API and so is less stable:
    # results = get_id_mapping_results_stream(link)

In [ ]:
for i in range(len(results['results'])):
    resulting = results['results'][i]['to']
    if resulting['entryType']== 'UniProtKB reviewed (Swiss-Prot)' and 'Homo sapiens' in str(resulting['organism']):
        uniprot_hpa.append(resulting['primaryAccession'])

We do an intersection of String Uniprot and Ensembl Uniprot in order to not have duplicates.

In [ ]:
noduplicates_Tissues= list(set(uniprot_hpa))
len(noduplicates_Tissues)

In [ ]:
with open(path_dir[directory8]+"/3.Tissues_ready_output.txt", 'w') as fp:
    for item in noduplicates_Tissues:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

In [ ]:
my_file_tissues = open(path_dir[directory8]+"/3.Tissues_ready_output.txt", "r")

content_tissues = my_file_tissues.read()

Tissues = content_tissues.split("\n")

my_file_tissues.close()

#### 1.4. Overlap

##### - Step 1. Import the list from HPA, Tissues and Bgee

In [ ]:
my_file_tissues = open(path_dir[directory8]+"/3.Tissues_ready_output.txt", "r")

content_tissues = my_file_tissues.read()

Tissues = content_tissues.split("\n")

my_file_tissues.close()

In [ ]:
my_file_bgee = open(path_dir[directory6]+"/1.Bgee_ready_output.txt", "r")

content_bgee = my_file_bgee.read()

Bgee = content_bgee.split("\n")

my_file_bgee.close()

In [ ]:
path_HPA = path_dir[directory7]+"/2.HPA_ready_output.csv"
HPA = pd.read_csv(path_HPA,delimiter='\t')

Due to UniProt, some values of the entry are duplicated and I do not know why. All the duplicated values
have the same info so we are going to eliminate one of them. This involves Tissues and Bgee lists.

##### - Step 2. Intersection between HPA-Tissues, HPA-Bgee, Bgee-Tissues

In [ ]:
HPA_UniprotCodes = HPA['Uniprot']

Tissues_UniprotCodes = Tissues

Bgee_UniprotCodes = Bgee

In [ ]:
ListCommon_HPA_Tissues = []

for i in HPA_UniprotCodes: # For each code in HPA 
    for j in Tissues_UniprotCodes: # For each code in Tissues
        if i ==j: # If both codes are equal we append it to the list of intersection HPA-Tissues
            ListCommon_HPA_Tissues.append(j)

In [ ]:
ListCommon_HPA_Bgee = []

for i in HPA_UniprotCodes: # For each code in HPA 
    for j in Bgee_UniprotCodes: # For each code in Tissues
        if i ==j: # If both codes are equal we append it to the list of intersection HPA-Tissues
            ListCommon_HPA_Bgee.append(j)

In [ ]:
ListCommon_Tissues_Bgee = []

for i in Bgee_UniprotCodes: # For each code in HPA 
    for j in Tissues_UniprotCodes: # For each code in Tissues
        if i ==j: # If both codes are equal we append it to the list of intersection HPA-Tissues
            ListCommon_Tissues_Bgee.append(j)

In [ ]:
print('Number of unique UniProt entries in HPA:', len(HPA_UniprotCodes))

print('Number of unique UniProt entries in Tissues:', len(Tissues_UniprotCodes))

print('Number of unique UniProt entries in Bgee:', len(Bgee_UniprotCodes))

print('---------------------------')

print('Intersection length HPA-Tissues:', len(ListCommon_HPA_Tissues))

print('Intersection length HPA-Bgee: ', len(ListCommon_HPA_Bgee))

print('Intersection length Bgee-Tissues: ', len(ListCommon_Tissues_Bgee))

In [ ]:
not_incommon_HPA_Tissues = []

intersection_list = list(ListCommon_HPA_Tissues)

for i in HPA_UniprotCodes:
    if i not in intersection_list:
        not_incommon_HPA_Tissues.append(i)
        
print('UniProts from HPA not in Tissues: ', len(not_incommon_HPA_Tissues))

In [ ]:
not_incommon_HPA_Bgee = []

intersection_list_2 = list(ListCommon_HPA_Bgee)

for i in HPA_UniprotCodes:
    if i not in intersection_list_2:
        not_incommon_HPA_Bgee.append(i)

print('UniProts from HPA not in Bgee: ', len(not_incommon_HPA_Bgee))

In [ ]:
not_incommon_Bgee_Tissues = []

intersection_list_3= list(ListCommon_Tissues_Bgee)

for i in Bgee_UniprotCodes:
    if i not in intersection_list_3:
        not_incommon_Bgee_Tissues.append(i)

print('UniProts from Bgee not in Tissues: ', len(not_incommon_Bgee_Tissues))

##### - Step 3. Intersection between HPA and Tissues and Bgee

In [ ]:
ListCommon_HPA_Tissues_Bgee = []

for i in ListCommon_HPA_Tissues:
    for j in Bgee_UniprotCodes: 
        if i ==j: 
            ListCommon_HPA_Tissues_Bgee.append(j)

In [ ]:
print('Intersection length HPA-Tissues-Bgee:', len(ListCommon_HPA_Tissues_Bgee))

print('Length HPA:', len(HPA_UniprotCodes))

print('Length Tissues:', len(Tissues_UniprotCodes))

print('Length Bgee:', len(Bgee_UniprotCodes))


In [ ]:
with open(path_dir[directory9]+"/IntersectionListCNS.txt", 'w') as fp:
    for item in ListCommon_HPA_Tissues_Bgee:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

In [ ]:
my_file_intCNS = open(path_dir[directory9]+"/IntersectionListCNS.txt", "r")

content_intCNS = my_file_intCNS.read()

IntersectionList = content_intCNS.split("\n")

my_file_intCNS.close()

In [ ]:
path_intersectionlist='~/Escritorio/TFG/1.LIST_PROTEINS/Isa_List_proteins/AllJupyters/1. Lists CNS/1.4.Overlap/IntersectionListCNS.csv'

IntersectionList_2= pd.read_csv(path_intersectionlist, sep = '\t')

In [ ]:
len(list(pd.unique(IntersectionList_2['Entry'])))

In [ ]:
len(ListCommon_HPA_Tissues_Bgee)

# ___________
# ___________ 

### 2. FILTERS

#### 2.1. Drug Targetable. Opentargets

##### A. Druggable with small molecule activity

We are going to get from pharos the datasets: 
    Tchem: 
    Tclin: 
        
This will give us the targets which have an Approved Drug and Ligand binding sites. 

We are going to get from Open Targets: 
    High-Quality Pocket: Target has a DrugEBIlity score of ≥ 0.7 
    
This will give us the targets whose scoring is bigger than 0.7.  The scoring is based on the conformational state of the protein structure that is analysed, if there is potentially an induced, or cryptic 'drugable' site. Finally, bear in mind that it is a statistical method, with some error, it is not meant to be definitive, but acts as a guide. -> http://chembl.github.io/drugebility-structure-based-component/

###### - Step 1. Collect data from Pharos

In [ ]:

path_pharos = path_dir[directory1]+"/PharosTCRD_UniProt_Mapping.tsv"

pharos = pd.read_csv(path_pharos,delimiter='\t')

pharos

In [ ]:
pharos_tchem_tclin = pharos[(pharos["TDL"] == 'Tchem') | (pharos["TDL"] == 'Tclin')]

In [ ]:
print('Length *unique* values of UniProt column: ', len(pd.unique(pharos_tchem_tclin['UniProt_accession'])), '. Length values of Uniprot column: ', len(pharos_tchem_tclin['UniProt_accession']))


In [ ]:
uniprots_pharos = list(pharos_tchem_tclin['UniProt_accession'])

##### - Step 2. Collect data from OpenTargets

In [ ]:
os.chdir(os.getcwd())

In [ ]:

# We open the data Open Targets gives us: 
#This data is the targets in general.

lista_pandas = []

for num in range(200): 
    if len(str(num)) == 1: 
        num_def = '0000'+str(num)
        
    if len(str(num)) == 2: 
        num_def = '000'+str(num)
        
    if len(str(num)) == 3: 
        num_def = '00'+str(num)
        
    
    path = path_dir[directory1]+'/targets/part-'+num_def+'-a58b02a9-0daa-4cf0-aac3-20428ba520c2-c000.json'
    
    with open(path, 'r') as file:
        raw = file.read()

    raw = raw.replace('\n',',')
    raw = '[' + raw[:-1] + ']'

    test_json = path_dir[directory1]+'/targets/test.json'
    with open(test_json, 'w') as file:
        file.write(raw)
    
    with open(test_json, 'r') as file:
        datos = json.load(file)
    
    df = pd.DataFrame(datos)
    lista_pandas.append(df)

lista_opentarget = pd.concat(lista_pandas)

lista_opentarget.head()

In [ ]:
# Filter by protein coding. 

proteincoding_opentarget = lista_opentarget[lista_opentarget['biotype'] == 'protein_coding']

# Almost all the proteins here are 'considered targetable'. This is inexact as OpenTargets includes all proteins
# and then it specifiys which one are targetable (druggable, etc.). This is seen in the column tractability. 


*OT: Filter by small molecules: High-Quality Pocket: Target has a DrugEBIlity score of ≥ 0.7*


In [ ]:
# We filter by these ones in the column tractabiliy: 

In [ ]:
tractability = proteincoding_opentarget['tractability'] # We extract the tractability column

In [ ]:
accepted_modalities = ['SM']
accepted_ids = ['Approved Drug', 'Advanced Clinical', 'Phase 1 Clinical', 'High-Quality Pocket']
druggable_SM_ligands = pd.DataFrame(columns = proteincoding_opentarget. columns. values) # New dataframe to append all 
                                                                                         # druggable by small molecule activity. 
count = 0
m = -1 # It will be the index of the dataframe
list_id = []

for target in tractability: # For each element (list of dictionaries) of the list tractability created in the above cell
    m += 1 
    
    if type(target) != list and pd.isnull(target): continue # To avoid Nan values that may do this loop fail.
        
    for entry in target: # For each dictionary
        if entry['modality'] not in accepted_modalities: # If the value of the modality key is not in the accepted modalities list above described: 
            continue # We skip the current iteration
        if entry['id'] not in accepted_ids: # If the value of the id key is not in the accepted ids list above described:
            continue # We skip the current iteration
            
        # If entry[modality] is in accepted_modalities  and entry['id'] is in accepted_ids:  
        if entry['value'] == True: # Then if the value of the value key is True: 
            idx = m
            ids = str(proteincoding_opentarget.iloc[[idx]]['id']).split()[1]#
            row_to_append = proteincoding_opentarget.iloc[[idx]] # We indicate which row has to be appended
            druggable_SM_ligands = pd.concat([druggable_SM_ligands,row_to_append])# We add the row to the dataframe we have created. 
            list_id.append(ids) # We append the value of the id column 
            break # We break the most internal loop in order to avoid duplicates. 

In [ ]:
print('Number of genes considered to be druggable (fact or potentially) by Small Molecules: ', len(druggable_SM_ligands)) 

In [ ]:
druggable_SM_ligands.head()

In [ ]:
# We check that we have in the above dataframe no duplicates of id
len(pd.unique(druggable_SM_ligands['id']))

In [ ]:
# We select the UniProt codes of the above table.

uniprots_OT = []
protein_id_OT = druggable_SM_ligands['proteinIds'] # We get the 'proteinIds' of each 'id'

for i in protein_id_OT: 
    for y in i: 
        if y['source'] =='uniprot_swissprot': # We select only those UniProt IDs that its source is swissprot. 
            uniprot = y['id']
            uniprots_OT.append(uniprot)

##### Step 3. Intersection between data from Pharos and OpenTargets 

In [ ]:
# We do the intersection between Pharos and Open Targets

uniprots_druggable = []

for i in uniprots_OT: # uniprots_OT contain unique uniprots values
    for j in uniprots_pharos: # uniprots_pharos contain unique uniprots values
        if i==j:
            uniprots_druggable.append(i)

In [ ]:
print('Number of UniProts from intersection between Pharos and Open Targets: ', len(uniprots_druggable))

In [ ]:
# We get a list of unique values: 

Uniprot_SM_pocket_ligand = list(set(uniprots_druggable))
print('Number of *unique* UniProts from intersection between Pharos and Open Targets: ', len(Uniprot_SM_pocket_ligand))

In [ ]:
# We load the CNS proteins list

my_file_intCNS = open(path_dir[directory9]+"/IntersectionListCNS.txt", "r")

content_intCNS = my_file_intCNS.read()

IntersectionList = content_intCNS.split("\n")

my_file_intCNS.close()

In [ ]:
# We see which proteins from the CNS protein list are targets; considering target
# as druggable by Small Molecule activity.

Common_OpenTarget_IntersectionList = []

for i in Uniprot_SM_pocket_ligand:
    for j in IntersectionList: 
        if i ==j: 
            Common_OpenTarget_IntersectionList.append(j)

In [ ]:
print('Number of entries (UniProts) in Open Targets filtered by druggable by SM:',len(Uniprot_SM_pocket_ligand))
print('Number of entries (UniProts) in the CNS protein list :',len(IntersectionList))
print('-----------')
print('Number of entries (UniProts) Open Targets-CNS protein list:',len(Common_OpenTarget_IntersectionList))

In [ ]:
with open(path_dir[directory1]+"/OT_druggable_SM.txt", 'w') as fp:
    for item in Common_OpenTarget_IntersectionList:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

#### B. Parkinson Diseases targets

##### - Step 1. Collect data from OpenTargets

In [ ]:

lista_pandas = []

for num in range(200): 
    if len(str(num)) == 1: 
        num_def = '0000'+str(num)
        
    if len(str(num)) == 2: 
        num_def = '000'+str(num)
        
    if len(str(num)) == 3: 
        num_def = '00'+str(num)
        
    
    path = path_dir[directory1]+'/associationByOverallDirect/part-'+num_def+'-af5f1d71-7e3c-4acd-ba55-90a325611413-c000.json'
    
    with open(path, 'r') as file:
        raw = file.read()

    raw = raw.replace('\n',',')
    raw = '[' + raw[:-1] + ']'

    test_json = path_dir[directory1]+'/associationByOverallDirect/test.json'
    with open(test_json, 'w') as file:
        file.write(raw)
    
    with open(test_json, 'r') as file:
        datos = json.load(file)
    
    df = pd.DataFrame(datos)
    lista_pandas.append(df)
    
lista_opentarget_diseases = pd.concat(lista_pandas)
print('Number of entries in the Open Targets Disease DB: ', len(lista_opentarget_diseases))

In [ ]:
# We filter by MONDO ontology: Parkinson Disease: MONDO_0005180

PD = lista_opentarget_diseases[lista_opentarget_diseases['diseaseId'] == 'MONDO_0005180']

print('Number of entries in the Open Targets DB filtered by Parkinson Disease: ', len(PD))

In [ ]:
PD.head()

In [ ]:
PD['score'].describe()

In [ ]:
# We are going to set a threshold value in order to have strong evidences that that target is for the Parkinson's Disease

# To do this, first we are going to normalize all the values 

list_scores = list(PD['score'])



In [ ]:
score_normscores_dict= {} # We create a dictionary because of the float problem
norm_scores =[] # Normalized scores will be inside here also

for score in list_scores: 
    
    norm_score = score/ max(list_scores)
    score_normscores_dict[score] = norm_score
    
    norm_scores.append(norm_score)
    

In [ ]:
import matplotlib.pyplot as plt

plt.figure(1)
plt.hist(norm_scores, bins=20)

plt.title('Histogram of frequency of normalized scores for Parkinsons Disease')
plt.ylabel('Number of score values')
plt.xlabel('Normalized score value')

plt.ylim(0,100)

From this plot we see that the majority of the norm scores are between ~0 and 0.2. This means that they have very little evidence because maybe the are few sources that have studied this target. 
We see that from 0.4 upwards there are some scores. We can consider that 0.5 in norm data is quite of reliable as  it is the half of the maximum score. 

In [ ]:
norm_scores_05 = [i for i in norm_scores if i > 0.5]

print('Number of scores for PD that have a normalized score bigger than 0.5: ', len(norm_scores_05))

In [ ]:
scores_05 = []

for score_dicti in score_normscores_dict: # For each normalized value of PD score
    score_dicti_norm = score_normscores_dict[score_dicti] 
    
    if score_dicti_norm in norm_scores_05: # If this value is in the list of scores above 0.5
        scores_05.append(score_dicti) # Append to a list where there will be the interesting ones

In [ ]:
len(scores_05)

In [ ]:
scores_05_unique = list(set(scores_05))

print('Number of unique scores above 0.5 of their normalized value: ', len(scores_05_unique))

In [ ]:
PD_scorefilter = pd.DataFrame(columns = PD. columns. values) # We create new dataframe to append those targets with
                                                            # a normalized score bigger than 0.5

for i in scores_05_unique: 
    row_to_append = PD[PD['score'] == i]
    PD_scorefilter = pd.concat([PD_scorefilter,row_to_append])

In [ ]:
print('Dataframe for Parkinsons Diease targets with a normalized score bigger than 0.5: ')
PD_scorefilter

In [ ]:
PD_scorefilter_id_2 = PD_scorefilter['targetId'] # We get the target IDs

In [ ]:
len(PD_scorefilter_id_2)

In [ ]:
#PD_scorefilter_id = PD_scorefilter['targetId'] # We get the target IDs

In [ ]:
uniprot_PD = []

for i in PD_scorefilter_id_2: 
    protein_ids = proteincoding_opentarget[proteincoding_opentarget['id'] == i]['proteinIds']
    for j in protein_ids: 
        for m in j:
            if m['source'] =='uniprot_swissprot': # We get the UniProt ID for the specific ENSG ID 
                uniprot = m['id']
                uniprot_PD.append(uniprot)
            if 'uniprot_swissprot' not in str(list(protein_ids)):
                print('These do not have a UniProt ID: ',i)

In [ ]:
len(uniprot_PD) # Two do not have a uniprot_swissprot source

#### C. Intersection between Pharos-OpenTargets Targets and OpenTargets Associations: Druggable with SM activity - Parkinson Diseases

In [ ]:
uniprot_druggable_PD = []

for i in Uniprot_SM_pocket_ligand: # We do the intersection between the targetable by druggable by Small Molecule activity 
    if i in uniprot_PD:  # and the targetable for Parkinsons Disease
        uniprot_druggable_PD.append(i) # The common ones, we append it to a list 
        

In [ ]:
len(uniprot_druggable_PD)

In [ ]:
Common_OpenTarget_IntersectionList = []

for i in uniprot_druggable_PD:
    for j in IntersectionList: 
        if i ==j: 
            Common_OpenTarget_IntersectionList.append(j)

In [ ]:
print('Number of UniProts Druggable by SM and PD :',len(uniprot_druggable_PD))
print('Number of UniProts from the CNS protein list:',len(IntersectionList))
print('Number of UniProts from CNS and Targetable:',len(Common_OpenTarget_IntersectionList))

#### D. Intersection between druggable with SM activity for PD and CNS protein list

##### Save list of CNS proteins that are targetable for SM for PD 

In [ ]:
with open(path_dir[directory1]+"/OT_PD_druggable_SM_correct.txt", 'w') as fp:
    for item in Common_OpenTarget_IntersectionList:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

# ___________

#### 2.2. Membrane. Uniprot

##### - Step 1. Import data from UniProt

We import the list from Uniprot including Transmembrane and Intermembrane proteins

In [ ]:
with gzip.open(path_dir[directory2]+'/stream') as f:

    Uniprot_Membrane= pd.read_csv(f,delimiter='\t')# Read the tsv. Tsv are limited by a t that's why it is called tsv.

In [ ]:
Uniprot_Membrane.head()

In [ ]:
my_file_intCNSOpt = open(path_dir[directory1]+"/OT_PD_druggable_SM_correct.txt", "r")

content_intCNSOpt = my_file_intCNSOpt.read()

IntersectionListOptarg = content_intCNSOpt.split("\n")

my_file_intCNSOpt.close()

##### - Step 2. Intersection between CNS target proteins and membrane proteins

In [ ]:
Common_MembIntermemb_IntersectionList = []

for i in Uniprot_Membrane['Entry']: # For each code of Uniprot_Membrane (Uniprot codes with membrane domains)
    for j in IntersectionListOptarg: # For each code of IntersectionList (Uniprot code of CNS list)
        if i ==j: # If they are equal
            Common_MembIntermemb_IntersectionList.append(j)

In [ ]:
print('Number of UniProts which are Membrane proteins:',len(Uniprot_Membrane))
print('Number of UniProts which are CNS Targetable proteins (with some filters):',len(IntersectionListOptarg))
print('Number of UniProts which are CNS Targetable Membrane proteins:',len(Common_MembIntermemb_IntersectionList))

##### - Step 3. Save the list for CNS target membrane proteins

In [ ]:
with open(path_dir[directory2]+"/List_CNS_DruggablePD_Mem_Uniprot_correct.txt", 'w') as fp:
    for item in Common_MembIntermemb_IntersectionList:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

# ______

#### 2.3. Length sequence > 100 aa. Uniprot

##### - Step 1. Import data

In [ ]:
my_file_memCNS = open(path_dir[directory2]+"/List_CNS_DruggablePD_Mem_Uniprot_correct.txt", "r")

content_memCNS = my_file_memCNS.read()

Membrane_CNS = content_memCNS.split("\n")

my_file_memCNS.close()

In [ ]:
print('CNS Targetable Membrane proteins: ',len(Membrane_CNS)-1) # Because the last character of the list is an empty space


In [ ]:
Membrane_CNS_UniProt_df = Uniprot_Membrane[Uniprot_Membrane['Entry'].isin(Membrane_CNS)]
Membrane_CNS_UniProt_df

##### - Step 2. Cut-off length aa

In [ ]:
# This is a list of the lengths of each protein
Length_aa = Membrane_CNS_UniProt_df['Length']

In [ ]:
# We get useful information of the Length column in order to set a cut-off
Length_aa.describe()

We create a frequency table to know which intervals are more frequent and examine the structure of the protein and see if it has an interesting one (different chains, etc) or a simple one (one helix alpha...) so it is not that much interesting for studying its dynamics.

In [ ]:
iw = (Length_aa.max()-Length_aa.min())/200


In [ ]:
Abs_frecuency, intervals = np.histogram(Length_aa, bins = 100)


In [ ]:
# Create dataframe
freq_table = pd.DataFrame(index = np.linspace(1,100,100), columns = ['start', 'end', 'class_marks','Frec_abs'])
# Assign the intervals
freq_table['start'] = intervals[:-1]
freq_table['end'] = intervals[1:]
# Calculate class marks
freq_table['class_marks'] = (freq_table['start'] + freq_table['end'])/2
# Assing Absolute frecuency
freq_table['Frec_abs'] = Abs_frecuency


In [ ]:
freq_table

In [ ]:
filter_length = Membrane_CNS_UniProt_df

# ____

#### 2.4. With PDB structure 
 When we have already done filter of length aa of the membrane-intersection cns

##### - Step 1. Filter those proteins that have PDB structure

In [ ]:
# We select the Uniprot codes ('Entry')
Uniprot_codes = filter_length['Entry'].to_list()

In [ ]:
# We load the document proportionated by lab of P.Aloy
path_uniprot_pdb_file = str(input("Write path where you have downloaded the file uniprot_pdb: "))
uniprot = pd.read_csv(path_uniprot_pdb_file+'/uniprot_pdb.tsv',delimiter='\t')

In [ ]:
# We do a table of those Uniprot_codes that are found in the SP_PRIMARY column of the file uniprot_pdb
table_PDBs_CNS = uniprot[uniprot['SP_PRIMARY'].isin(Uniprot_codes)]

In [ ]:
# We create the columns of the table
table_PDBs_CNS.columns=['Uniprot','PDB']

In [ ]:
# We add the values
table_PDBs_CNS['gene_name']=[filter_length[filter_length['Entry']==x]['Gene Names'].to_list()[0] for x in table_PDBs_CNS['Uniprot'].to_list()]

In [ ]:
# This is the final table where we have all the PDBs codes for each Uniprot belonging to CNS, membrane and length > 100 aa
table_PDBs_CNS

In [ ]:
print('We have: ',len(filter_length),' CNS Targetable Membrane proteins\nWe have: ',len(table_PDBs_CNS),' proteins with PDB structure')

In [ ]:
# We now extract all the PDBs codes that we have: 

PDBs_CNS_possibleduplicates = [] # There may be some duplicates.
table_PDB = list(table_PDBs_CNS['PDB']) 

for i in table_PDB: 
    if ';' in i: 
        for item in i.split(";"):# We split those values that are in the same row.
            PDBs_CNS_possibleduplicates.append(item)      
    else: 
        PDBs_CNS_possibleduplicates.append(i) #If there are not values in the same row, then we append it as it is.

In [ ]:
# We now get rid of the duplicates

PDBs_CNS = []

for item in PDBs_CNS_possibleduplicates:
    if item not in PDBs_CNS:
        PDBs_CNS.append(item)

In [ ]:
print('Number of PDBs that are from the CNS, Targets, Membrane,length>100, possible dupli:  ',len(PDBs_CNS_possibleduplicates),'PDBs')
print('Number of PDBs that are from the CNS, Targets, Membrane:  ',len(PDBs_CNS),'PDBs')

In [ ]:
# Length of the table
print('Total number of proteins from CNS, Targets, Membrane, length>100, PDB structure, possidupli:',len(table_PDBs_CNS), 'proteins')
print('Number of PDBs that are from the CNS, Targets, Membrane, length>100, possible dupli:  ',len(PDBs_CNS_possibleduplicates),'PDBs')
print('Total number of PDBs from CNS, Targets, Membrane, >100 aa proteins, without dupli: ', len(PDBs_CNS), 'PDBs')

In [ ]:
table_PDBs_CNS.to_csv(path_dir[directory4]+'/List_CNS_DruggablePD_Mem_100aa_PDB_Uniprot_correct.csv')


#### 2.4.1. Information about PDB structure

##### Split PDBs and make a table of keys: Uniprot and PDB

In [ ]:
table_PDBs_CNS

In [ ]:
Uniprot_id_dict = [] # We create a list of the multiple Uniprot for each PDB that exists

for PDB_id in list(table_PDBs_CNS['PDB']): # For each PDB (can be multiple PDBs)
    count_char = PDB_id.count(';')+1 # We count how many PDBs are in that string, the codes are split with ;
    
    idx = table_PDBs_CNS[table_PDBs_CNS['PDB'] == PDB_id].index.tolist() # We calculate the index of that PDB
    
    if PDB_id ==  list(table_PDBs_CNS.loc[idx[0]])[1]: # If the PDB string is equal to the value of PDBs in the row of the index
        Unid_multiple = (table_PDBs_CNS.loc[idx[0]][0],)*count_char # Then we multiply the id of Uniprot the number of times of PDB codes in that string
        Uniprot_id_dict.append(Unid_multiple) # We save the value in the list

In [ ]:
Uniprot_id_dict_nottuple = []

for Uniprot_id_dict_tuple in Uniprot_id_dict: 
    for Uniprot_id_indi in Uniprot_id_dict_tuple: 
        Uniprot_id_dict_nottuple.append(Uniprot_id_indi)
    

In [ ]:
genename_dict = [] # We create a list of the multiple genename for each PDB that exists

for PDB_id in list(table_PDBs_CNS['PDB']): # For each PDB (can be multiple PDBs)
    count_char = PDB_id.count(';')+1 # We count how many PDBs are in that string, the codes are split with ;
    
    idx = table_PDBs_CNS[table_PDBs_CNS['PDB'] == PDB_id].index.tolist() # We calculate the index of that PDB
    
    if PDB_id ==  list(table_PDBs_CNS.loc[idx[0]])[1]: # If the PDB string is equal to the value of PDBs in the row of the index
        genename_multiple = (table_PDBs_CNS.loc[idx[0]][2],)*count_char # Then we multiply the id of genename the number of times of PDB codes in that string
        genename_dict.append(genename_multiple) # We save the value in the list
        

In [ ]:
genename_dict_nottuple = []

for genename_dict_tuple in genename_dict: 
    for genename_indi in genename_dict_tuple: 
        genename_dict_nottuple.append(genename_indi)
    

In [ ]:
len(Uniprot_id_dict_nottuple),len(PDBs_CNS_possibleduplicates),len(genename_dict_nottuple)

In [ ]:
d = {'Uniprot':Uniprot_id_dict_nottuple,'PDB':PDBs_CNS_possibleduplicates,'gene_name':genename_dict_nottuple}
table_PDBs = pd.DataFrame.from_dict(d)

In [ ]:
table_PDBs

In [ ]:
len(table_PDBs['PDB'].unique())

##### API to get more info

To get the information about the **experimental method** of each PDB. 

In [ ]:
import json
import urllib.request
from typing import List

# Given a pdb Id, get its Experimental Obtaining Method

def pdb_info_exp (pdb_id : str) -> List[str]:
    # Request the MMB service to retrieve pdb data
    request_url = 'https://mmb.irbbarcelona.org/api/pdb/' + pdb_id + '/entry'
    try:
        with urllib.request.urlopen(request_url) as response:
            parsed_response = json.loads(response.read().decode("utf-8"))
    # If the accession is not found in the PDB then we can stop here
    except urllib.error.HTTPError as error:
        if error.code == 404:
            return None
        else:
            raise ValueError('Something went wrong with the PDB request: ' + request_url)
    # Get the uniprot accessions
    pdb_expType = parsed_response['expType'] # We get the Experimental Method
    
    return pdb_expType

To get the information about the **experimental resolution** of each PDB. 

In [ ]:
import json
import urllib.request
from typing import List

# Given a pdb Id, get its method resolution

def pdb_info_resol (pdb_id : str) -> List[str]:
    # Request the MMB service to retrieve pdb data
    request_url = 'https://mmb.irbbarcelona.org/api/pdb/' + pdb_id + '/entry'
    try:
        with urllib.request.urlopen(request_url) as response:
            parsed_response = json.loads(response.read().decode("utf-8"))
    # If the accession is not found in the PDB then we can stop here
    except urllib.error.HTTPError as error:
        if error.code == 404:
            return None
        else:
            raise ValueError('Something went wrong with the PDB request: ' + request_url)
    # Get the uniprot accessions
    pdb_resol= parsed_response['resol'] # We get the resolution 
    return pdb_resol

To get the information about the **information about chains: type, sequence, fragments** of each PDB. 

In [ ]:
import json
import urllib.request
from typing import List

# Given a pdb Id, gets information of each of its chains. 

def pdb_info_chains (pdb_id : str) -> List[str]:
    # Request the MMB service to retrieve pdb data
    request_url = 'https://mmb.irbbarcelona.org/api/pdb/' + pdb_id + '/entry'
    try:
        with urllib.request.urlopen(request_url) as response:
            parsed_response = json.loads(response.read().decode("utf-8"))
    # If the accession is not found in the PDB then we can stop here
    except urllib.error.HTTPError as error:
        if error.code == 404:
            return None
        else:
            raise ValueError('Something went wrong with the PDB request: ' + request_url)
    
    # Get the uniprot accessions
    if parsed_response['chainIds'] != None: # If there are ChainIds: 
        
        chains = [ chain[-1] for chain in parsed_response['chainIds'] ] 
        # We only stay with the last letter of the chain because it indicates the chain type
        

        id_chains = [ chain['_id'] for chain in parsed_response['chains'] ]
        # We get all the id of the chain 
        
        sequence_uniprot = [ chain['sequence'] for chain in parsed_response['chains']]
        # We get the sequence of Uniprot of each chaine if each PDB has different chains. 
        
        
        # To get the fragments we do not do a compressed loop: 
        fragments_all = [] # We create a list to put all the fragments of all chains here
        
        
        for chain in parsed_response['chains']: # We get the fragments of each chain
            if chain['PDBSequence'] != None: # as long as the PDBSequence exists because if not, we would not have fragments
                fragments = chain['PDBSequence']['fragments'] 
                fragments_all.append(fragments)

                
                
        # We want a dictionary with: being the key the chain type ['A'], and the values all the information of
        #this chain for this PDB ex. sequence['AGJEROW'], fragments ['1-22, 133-159']. 
        lista_dicti = []
        
        
        for idx in (range(len(chains))):
            if chain['PDBSequence'] != None: # as long as it exists a PDBSequence
                print(id_chains[idx])
                dicti = {chains[idx]:[sequence_uniprot[idx],fragments_all[idx]]}
                lista_dicti.append(dicti)

        return lista_dicti
    
    else:
        return None


To get the information about the **Chain ID (type)** of each PDB. 

In [ ]:
import json
import urllib.request
from typing import List

# Given a pdb Id, get its identifiers chains: 

def pdb_chains (pdb_id : str) -> List[str]:
    # Request the MMB service to retrieve pdb data
    request_url = 'https://mmb.irbbarcelona.org/api/pdb/' + pdb_id + '/entry'
    try:
        with urllib.request.urlopen(request_url) as response:
            parsed_response = json.loads(response.read().decode("utf-8"))
    # If the accession is not found in the PDB then we can stop here
    except urllib.error.HTTPError as error:
        if error.code == 404:
            return None
        else:
            raise ValueError('Something went wrong with the PDB request: ' + request_url)
    
    # Get the uniprot accessions
    if parsed_response['chainIds'] != None: 
        
        chains = [ chain[-1] for chain in parsed_response['chainIds'] ]
        #Here we only get the last letter of the chain id because faster than mining the previous dictionary. 

        return chains
    
    else:
        return None


We run each function. 

In [ ]:
pdb_info_exp_list = []

for PDB_id in table_PDBs['PDB']: 
    if PDB_id not in pdb_info_exp_list:
        pdb_info_api = pdb_info_exp(PDB_id)
        print(pdb_info_api)
        pdb_info_exp_list.append(pdb_info_api)

In [ ]:
pdb_info_resol_list = []

for PDB_id in table_PDBs['PDB']:
    if PDB_id not in pdb_info_resol_list:
        pdb_info_resol_api = pdb_info_resol(PDB_id)
        print(pdb_info_resol_api)
        pdb_info_resol_list.append(pdb_info_resol_api)

In [ ]:
pdb_info_chains_list = []

for PDB_id in table_PDBs['PDB']: 
    pdb_info_chain_api = pdb_info_chains(PDB_id)
    print(pdb_info_chain_api)
    pdb_info_chains_list.append(pdb_info_chain_api)

In [ ]:
pdb_chains_list = []

for PDB_id in table_PDBs['PDB']: 
    pdb_chain_api = pdb_chains(PDB_id)
    print(pdb_chain_api)
    pdb_chains_list.append(pdb_chain_api)

We create the table of all the things above: 

In [ ]:
data = {'Uniprot':Uniprot_id_dict_nottuple,'PDB':PDBs_CNS_possibleduplicates,'Chains':pdb_chains_list,'Info Chains: chain, seq, fragments':pdb_info_chains_list,'Experimental method':pdb_info_exp_list,'Experimental Resolution':pdb_info_resol_list,'gene_name':genename_dict_nottuple}
table_proteins_PDB_CNS_membrane_l100 = pd.DataFrame.from_dict(data)

In [ ]:
table_proteins_PDB_CNS_membrane_l100

In [ ]:
os.chdir(working_directory + '/Workflow_List_Proteins_List')

In [ ]:
table_proteins_PDB_CNS_membrane_l100.to_csv('PD_0_NOT_and_IN_MEMPROTMD_table_proteins_PDB_CNS_membrane_l100_correct.csv', index=False)

In [ ]:
table_proteins_PDB_CNS_membrane_l100 = pd.read_csv('PD_0_NOT_and_IN_MEMPROTMD_table_proteins_PDB_CNS_membrane_l100_correct.csv')

##### Review they are from CNS

We are going to review it with the HPA brain database, although Bgee and Tissues also would work. 

In [ ]:

HPA_raw = pd.read_csv(path_dir[directory7]+'/NOT.tsv',delimiter='\t')


In [ ]:
Uniprot_HPA = HPA_raw['Uniprot']

In [ ]:
Uniprot_df_notunique = table_proteins_PDB_CNS_membrane_l100['Uniprot']

In [ ]:
def unique_values(x):
    return list(dict.fromkeys(x))

Uniprot_df_unique = unique_values(table_proteins_PDB_CNS_membrane_l100['Uniprot'])


In [ ]:
n = 0
for i in Uniprot_df_unique:
    if i in list(Uniprot_HPA):
        n = n+1

In [ ]:
print('If the workflow was well done, all the proteins that we have in our list df should be from the CNS,and therefore\nshould be in the HPA')
print()
print('This means that the length of the unique Uniprot values of our list should be equal to the ones of this list into\nthe HPA brain database ')

In [ ]:
print('Number of Uniprot_df_unique: ',len(Uniprot_df_unique),'\nNumber of proteins found in HPA: ',n)

# ______

#### 2.5. In MemProtMD

##### API to know all info contained in MemprotMD

In [ ]:
MEMPROTMD_ROOT_URI = "http://memprotmd.bioch.ox.ac.uk/"

In [ ]:
# Definition of function to collect all the MemProtMD simulations

def get_all_memprotmd_simulations():
    return requests.post(MEMPROTMD_ROOT_URI + "api/simulations/all").json()

get_all_memprotmd_simulations()[0]

In [ ]:
all_memprotmd_simulations = pd.DataFrame.from_records(get_all_memprotmd_simulations())

In [ ]:
all_memprotmd_simulations

In [ ]:
# Codes of PDB that are in the database MemProtMD
pdb_codes_memprotmd = all_memprotmd_simulations['accession']

In [ ]:
# We now have to import the list of PDBs of Central Nervous System

In [ ]:
path_PDB_CNS = path_dir[directory4]+'/List_CNS_DruggablePD_Mem_100aa_PDB_Uniprot_correct.csv'
PDB_CNS = pd.read_csv (path_PDB_CNS,delimiter=',')

In [ ]:
PDB_CNS.head()

In [ ]:
# We collect all the PDBs of the CNS, separating those that are in the same row also

PDBs_CNS_possibleduplicates = []
table_PDB = list(PDB_CNS['PDB'])

for i in table_PDB: 
    if ';' in i: 
        for item in i.split(";"):
            PDBs_CNS_possibleduplicates.append(item)      
    else: 
        PDBs_CNS_possibleduplicates.append(i)

In [ ]:
# We remove duplicates PDBs of the CNS

PDBs_CNS = []
for item in PDBs_CNS_possibleduplicates:
    if item not in PDBs_CNS:
        PDBs_CNS.append(item)

In [ ]:
len(PDBs_CNS)

In [ ]:
list_codes_PDB_CNS = list(PDB_CNS['PDB'])

In [ ]:
# We do the intersection between the MemProtMD codes and the CNS codes of PDB. There may be some duplicates

codes_MemProtMD_CNS_possibleduplicates =  []

for PDB_MemProtMD in pdb_codes_memprotmd:
    for j in list_codes_PDB_CNS:
        if PDB_MemProtMD in j:
            codes_MemProtMD_CNS_possibleduplicates.append(PDB_MemProtMD)


In [ ]:
len(codes_MemProtMD_CNS_possibleduplicates)

In [ ]:
# We remove the duplicates of the previous list
# So we have the codes of the MemProtMD of the CNS 

codes_MemProtMD_CNS = set(codes_MemProtMD_CNS_possibleduplicates)
len(codes_MemProtMD_CNS)

In [ ]:
print('Number of PDBs of CNS with all filters, no dupli (unique):',len(PDBs_CNS),'\nNumber of PDBs of CNS with all filters in MemProtMD (unique)',len(codes_MemProtMD_CNS))

##### Download the data of MemProt relative to CNS' PDBs already filtered

We do it with APIs. From PDB to MemProtMD

In [ ]:
import json
import urllib.request
from typing import List


def get_all_zip_memprotmd_simulations (pdb_id : str):

    # Request the MMB service to retrieve pdb data
    request_url = 'http://memprotmd.bioch.ox.ac.uk/data/memprotmd/simulations/'+pdb_id +'_default_dppc/files/run/at.zip'
    
    try:
        with urllib.request.urlopen(request_url) as response:
            parsed_response = requests.get(request_url, allow_redirects=True)
            # Write the content to disk
            open(path_dir[directory5]+'/'+pdb_id+'_default_dppc-atomistic-simulation.zip', 'wb').write(parsed_response.content)
            
    # If the accession is not found in the PDB then we can stop here
    except urllib.error.HTTPError as error:
        if error.code == 404:
            return None
        else:
            raise ValueError('Something went wrong with the PDB request: ' + request_url)
    except Exception as error:
        print(request_url)
        print(error)            

In [ ]:

for pdb_id in codes_MemProtMD_CNS:
    output_file = path_dir[directory5]+'/'+pdb_id+'_default_dppc-atomistic-simulation.zip'
    output_file_extra = path_dir[directory5]+'/'+pdb_id+'_default_dppc-atomistic-simulation_extra.zip'
    if not os.path.exists(output_file) and not os.path.exists(output_file_extra):
        print('Requesting ' + pdb_id)
        get_all_zip_memprotmd_simulations(pdb_id)
        

In [ ]:
len(codes_MemProtMD_CNS)

In [ ]:
path_PDB_download = path_dir[directory5]
path_PDB_download_items = os.listdir(path_PDB_download)

for PDB_downloaded in path_PDB_download_items:
    
    split_string = PDB_downloaded.split('_')
    name = split_string[0]
    
    path_PDB =path_PDB_download+'/'+PDB_downloaded
    
    
    if os.path.getsize(path_PDB) != 0: # If size of the file downloaded is different to 0
        
        if name not in codes_MemProtMD_CNS and len(name) ==4: # If the name is not in the codes we have collected from CNS and filters

            #os.remove(path_PDB) # We remove the file. This is to be sure that the downloaded ones are from the CNS and filters list. 
            print('Does not make sense the downloaded of this PDB, it is not from our interest list (CNS and filters): ', name)
            
            
    else: # If size of the file downloaded is equal to 0, it was not correctly downloaded and we remove it: 
        
        print('Not correctly downloaded',name)
        os.remove(path_PDB) 
    

In [ ]:
PDBs_MemProtMD_correctlydownloaded = [i for i in os.listdir(path_PDB_download) if i != 'ToStudy_NonRepeatedSequences']

In [ ]:
len(PDBs_MemProtMD_correctlydownloaded)

#### 2.5.1. Select those that are in MemProtMD

In [ ]:
len(PDBs_MemProtMD_correctlydownloaded)

In [ ]:
PDBs_MemProtMD_correctlydownloaded_id = []

for pdb_at in PDBs_MemProtMD_correctlydownloaded:
    pdb_id = pdb_at[0:4]
    PDBs_MemProtMD_correctlydownloaded_id.append(pdb_id)
    

In [ ]:
UniprotMD_list = []
PDBMD_list = []
ChainsMD_list = []
InfoChainsMD_list = []
MethodMD_list = []
ResolutionMD_list = []
gene_nameMD_list = []

for idx in range(len(PDBs_MemProtMD_correctlydownloaded_id)):
    if PDBs_MemProtMD_correctlydownloaded_id[idx] in list(table_proteins_PDB_CNS_membrane_l100['PDB']): 
        
        UniprotMD_indi = list(table_proteins_PDB_CNS_membrane_l100[table_proteins_PDB_CNS_membrane_l100['PDB'] == PDBs_MemProtMD_correctlydownloaded_id[idx]][table_proteins_PDB_CNS_membrane_l100.columns[0]])
        PDBMD_indi = list(table_proteins_PDB_CNS_membrane_l100[table_proteins_PDB_CNS_membrane_l100['PDB'] == PDBs_MemProtMD_correctlydownloaded_id[idx]][table_proteins_PDB_CNS_membrane_l100.columns[1]])
        ChainsMD_indi=list(table_proteins_PDB_CNS_membrane_l100[table_proteins_PDB_CNS_membrane_l100['PDB'] == PDBs_MemProtMD_correctlydownloaded_id[idx]][table_proteins_PDB_CNS_membrane_l100.columns[2]])
        InfoChainsMD_indi = list(table_proteins_PDB_CNS_membrane_l100[table_proteins_PDB_CNS_membrane_l100['PDB'] == PDBs_MemProtMD_correctlydownloaded_id[idx]][table_proteins_PDB_CNS_membrane_l100.columns[3]])
        MethodMD_indi = list(table_proteins_PDB_CNS_membrane_l100[table_proteins_PDB_CNS_membrane_l100['PDB'] == PDBs_MemProtMD_correctlydownloaded_id[idx]][table_proteins_PDB_CNS_membrane_l100.columns[4]])
        ResolutionMD_indi = list(table_proteins_PDB_CNS_membrane_l100[table_proteins_PDB_CNS_membrane_l100['PDB'] == PDBs_MemProtMD_correctlydownloaded_id[idx]][table_proteins_PDB_CNS_membrane_l100.columns[5]])
        gene_nameMD_indi = list(table_proteins_PDB_CNS_membrane_l100[table_proteins_PDB_CNS_membrane_l100['PDB'] == PDBs_MemProtMD_correctlydownloaded_id[idx]][table_proteins_PDB_CNS_membrane_l100.columns[6]])
        
        
        UniprotMD_list.append(UniprotMD_indi)
        PDBMD_list.append(PDBMD_indi)
        ChainsMD_list.append(ChainsMD_indi)
        InfoChainsMD_list.append(InfoChainsMD_indi)
        MethodMD_list.append(MethodMD_indi)
        ResolutionMD_list.append(ResolutionMD_indi)
        gene_nameMD_list.append(gene_nameMD_indi)

In [ ]:
ready_UniprotMD_list = [item for sublist in UniprotMD_list for item in sublist]
ready_PDBMD_list = [item for sublist in PDBMD_list for item in sublist]
ready_ChainsMD_list = [item for sublist in ChainsMD_list for item in sublist]
ready_InfoChainsMD_list = [item for sublist in InfoChainsMD_list for item in sublist]
ready_MethodMD_list = [item for sublist in MethodMD_list for item in sublist]
ready_ResolutionMD_list = [item for sublist in ResolutionMD_list for item in sublist]
ready_gene_nameMD_list = [item for sublist in gene_nameMD_list for item in sublist]

In [ ]:
data_MemProt= {'Uniprot':ready_UniprotMD_list,'PDB':ready_PDBMD_list,'Chains':ready_ChainsMD_list,'Info Chains: chain, seq, fragments':ready_InfoChainsMD_list,'Experimental method':ready_MethodMD_list,'Experimental Resolution':ready_ResolutionMD_list,'gene_name':ready_gene_nameMD_list}
table_proteins_PDB_CNS_membrane_l100_MemProtMD = pd.DataFrame.from_dict(data_MemProt)

In [ ]:
table_proteins_PDB_CNS_membrane_l100_MemProtMD.sort_values(by = ['Uniprot','PDB'])

In [ ]:
Uniprot_unique = table_proteins_PDB_CNS_membrane_l100_MemProtMD['Uniprot'].unique()

uniprots_length = []
length_list = []

for i in Uniprot_unique: 
    
    length = len(table_proteins_PDB_CNS_membrane_l100_MemProtMD[table_proteins_PDB_CNS_membrane_l100_MemProtMD['Uniprot'] == i])
    uniprots_length.append(i)
    length_list.append(length)

In [ ]:
max_leng_idx = length_list.index(max(length_list))
print(max(length_list))

In [ ]:
uniprots_length[max_leng_idx]

In [ ]:
os.chdir(working_directory + '/Workflow_List_Proteins_List')
table_proteins_PDB_CNS_membrane_l100_MemProtMD.to_csv('1_DruggablePD_MEMPROTMD_table_proteins_PDB_CNS_membrane_l100_correct_2.csv', index=False)

#### 2.5.2. Select those not in MemProtMD

In [ ]:
PDB_id_Memprot_copy = table_proteins_PDB_CNS_membrane_l100_MemProtMD['PDB']
Uniprot_id_Memprot_copy = table_proteins_PDB_CNS_membrane_l100_MemProtMD['Uniprot']

In [ ]:
table_proteins_PDB_CNS_membrane_l100_notMemProt = table_proteins_PDB_CNS_membrane_l100

for PDB_id_Memprot,Uniprot_id_Memprot in zip(PDB_id_Memprot_copy,Uniprot_id_Memprot_copy): 
    table_proteins_PDB_CNS_membrane_l100_notMemProt.drop(table_proteins_PDB_CNS_membrane_l100_notMemProt[(table_proteins_PDB_CNS_membrane_l100_notMemProt['PDB'] == PDB_id_Memprot) & (table_proteins_PDB_CNS_membrane_l100_notMemProt['Uniprot'] == Uniprot_id_Memprot)].index,inplace=True)
    

In [ ]:
table_proteins_PDB_CNS_membrane_l100_notMemProt.sort_values(by = ['Uniprot','PDB'])

In [ ]:
os.chdir(working_directory + '/Workflow_List_Proteins_List')
table_proteins_PDB_CNS_membrane_l100_notMemProt.to_csv('2_DruggablePD_NOT_IN_MEMPROTMD_table_proteins_PDB_CNS_membrane_l100_correct_2.csv', index=False)

#### 2.5.3. Only PDB table (not Uniprot) and non-repeated

In [ ]:
list1 = pd.read_csv('1_DruggablePD_MEMPROTMD_table_proteins_PDB_CNS_membrane_l100_correct_2.csv')

In [ ]:
list1.drop('Uniprot', inplace=True, axis=1)

In [ ]:
list1_noduplisPDB = list1.drop_duplicates(subset=['PDB'])

In [ ]:
os.chdir(working_directory + '/Workflow_List_Proteins_List')
list1_noduplisPDB.to_csv('1_DruggablePD_nodupliPDB_IN_MEMPROTMD_table_proteins_PDB_CNS_membrane_l100_correct_2.csv', index=False)

In [ ]:
len(list1_noduplisPDB)

In [ ]:
list2 = pd.read_csv('2_DruggablePD_NOT_IN_MEMPROTMD_table_proteins_PDB_CNS_membrane_l100_correct_2.csv')

In [ ]:
list2.drop('Uniprot', inplace=True, axis=1)

In [ ]:
list2_noduplisPDB = list2.drop_duplicates(subset=['PDB'])

In [ ]:
len(list2_noduplisPDB)

In [ ]:
os.chdir(working_directory + '/Workflow_List_Proteins_List')
list2_noduplisPDB.to_csv('2_DruggablePDB_nodupliPDB_NOT_IN_MEMPROTMD_table_proteins_PDB_CNS_membrane_l100_correct_2.csv', index=False)

### 3. PDBs not equal-selection

##### From PD and Memprot

In [ ]:
table_proteins_PDB_CNS_membrane_l100_MemProtMD

In [ ]:
uniprots_memprot = table_proteins_PDB_CNS_membrane_l100_MemProtMD['Uniprot'].unique()

In [ ]:
PDBs_memprot_for_uniqueUniprot =[]

for i in uniprots_memprot: 
    PDBs_ = list(table_proteins_PDB_CNS_membrane_l100_MemProtMD[table_proteins_PDB_CNS_membrane_l100_MemProtMD['Uniprot'] ==i]['PDB'])
    PDBs_memprot_for_uniqueUniprot.append(PDBs_)

In [ ]:
PDBs_memprot_for_uniqueUniprot

In [ ]:

# Given a pdb id return the explicit sequence
def get_pdb_seq (pdb_id : str) -> list:
    # Request the MMB service to retrieve pdb data
    request_url = 'https://mmb.irbbarcelona.org/api/pdb/' + pdb_id + '/entry'
    try:
        with urllib.request.urlopen(request_url) as response:
            parsed_response = json.loads(response.read().decode("utf-8"))
    # If the accession is not found in the PDB then we can stop here
    except urllib.error.HTTPError as error:
        if error.code == 404:
            print('NOT FOUND')
            return {pdb_id:None}
        else:
            raise ValueError('Something went wrong with the PDB request: ' + request_url)
    
    # If there are no chains we are done
    if parsed_response['chainIds'] == None:
        print('NO CHAINS ' + pdb_id)
        return {pdb_id:None}

    seqs = [] # Conjunto de secuencia de PDB para cada cadena. [1 --> seq cadena A, 2--> seq cadena B...]
    for chain in parsed_response['chains']:
        if chain['PDBSequence'] != None: 
            #chain_id = chain['_id'][-1]
            seq = chain['PDBSequence']['sequence'] # Cogemos la secuencia del PDB de cada cadena
            seqs.append(seq)

    return {pdb_id:seqs} # Devuelve lista de secuencias. 

In [ ]:
# Set a special iteration system
# Return one value of the array and a new array with all other values for each value
def otherwise (values : list) -> Generator[tuple, None, None]: # Pondremos la lista (listo) como argumento
    for v, value in enumerate(values): # de cada listi dentro de esta lista, seleccionamos su índice dentro de la lista y tal cual la listi. 
        others = values[v+1:] # Seleccionamos todas aquellas listis dentro de la lista diferentes a la iterada (diferentes a la del PDB). 
        yield value, others # Devolvemos la listi iterada y el resto de listis. 

In [ ]:
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

In [ ]:
pdb_to_study = []
pdbs_disc = []

for sample in PDBs_memprot_for_uniqueUniprot: 
    
    #print('SAMPLE: ',sample)
    #print([ get_pdb_seq(s) for s in sample ])
    
    seqs = [ get_pdb_seq(s).get(s) for s in sample ] # Para cada PDB, hacemos una lista de listas de cada cadena. Esto para cada UniProt
    #print('SECUENCIA ????',seqs)
    seqs = [ s for s in seqs if s ] # Descartamos aquellas que son None (con falsy). 
    #print('SECUENCIA',seqs)


    # seqs es una lista (listo) de listas (listi) donde dentro de esta listi cada elemento es una cadena de ese PDB y cada
    # elemento de la lista (listo) es un PDB distinto. 

    
    discarded = [] # Lista de secuencias descartadas

    for i, (seq_set, other_seq_sets) in enumerate(otherwise(seqs)): # Aplicamos la funcion otherwise a todos los eleementos
                                                                    # de la lista seqs. 
                                                                    # Seq_set será una lista de listas de todas las cadenas de ese PDB
                                                                    # o si tiene una cadena será solo esa cadena del PDB
                                                                    # Other_seq_sets será una lista de listas de todas las otras cadenas del resto de PDBs
                                                                    # o si solo tiene 1 cadena ese PDB, una lista de esa cadena en los distintos PDBs. 

        if i in discarded: # Si el índice de esa cadena ya lo hemos descartado antes, pues nada, continua. 
            continue

        #1r filtro de longitud
        for j, other_seq_set in enumerate(other_seq_sets): # Para cada 'other' secuencia
            # Compare sequences one by one
            if len(seq_set) != len(other_seq_set): # Si la longitud de la cadena/s seleccionada es diferent a la del resto(s) de cadena, entonces ya queda
                                                    # sabemos que serán diferentes entre ellas, entonces coninúamos. 
                continue
            mismatch = False # Atribuímos el valor False a la variable mismatch


            for seq in seq_set: # Para cada secuencia (cada cadena) de ese PDB 
                if seq not in other_seq_set: # si la secuencia no se encuentra en la lista de las otras secuencias
                    mismatch = True # Le damos el valor de True a la variable mismatch
                    break # y dejamos el bucle porque si no se encuentra eso es que es diferente y por lo tanto nos interesa. 
                          # nos interesan aquellos PDBs que tienen alguna de sus cadenas diferentes al resto. 
            if mismatch: # si el mismatch es True
                continue # se sigue
            # Current seqs are repeated
            discarded.append(i+j+1) # en el caso de que la len sea igual, y la secuencia se encuentre en la lista de las otras
                                    # añadimos el índice del PDB 


    unique_seqs = [ s for i, s in enumerate(seqs) if i not in discarded ] # de la lista de secuencias, hacemos 
                                                                            # una lista de aquellas que no han sido 
                                                                            # descartadas
    discard_seq = [s for i, s in enumerate(seqs) if i in discarded ]
    
    dic =[ get_pdb_seq(s) for s in sample ] # definimos el diccionario de antes. Es una lista de muchos diccionarios

    result = {} # Hacemos que la lista de diccionarios sea 1D, es decir, una única lista de 1 diccionario
    for d in dic:
        result.update(d)
     
    
    pdbs_with_same_seq= [] 

    for i in unique_seqs: #Para cada secuencia única de PDB que hemos encontrado
        keys = get_keys_from_value(result,i) # De esta secuencia, nos devuelve el PDB (la key del diccionario). 
                                            # Tenemos secuencias que son únicas pertenecientes a un solo PDB
                                            # Pero también tenemos aquella secuencia que se repite en varios PDBs
                                            # Si varios PDBs la tienen, solo la queremos estudiar 1 vez, 
                                            # por eso haremos lo siguiente: 
        pdbs_with_same_seq.append(keys)
    

    for pdb in pdbs_with_same_seq:
        pdb_stu = pdb[0] # De la lista de listas de PDBs, cogemos el primero. Es indiferent cuál PDB cogemos porque 
                        # si se repite la secuencia en varios, será el mismo PDB pero con distinto nombre. 
        pdb_to_study.append(pdb_stu)
        

    
    
    for i in discard_seq:
        keys = get_keys_from_value(result,i)
        pdbs_disc.append(keys)
        


In [ ]:
pdbs_disc

In [ ]:
pdb_to_study

In [ ]:
len(pdb_to_study)

In [ ]:
pdb_to_study_unique = list(set(pdb_to_study))

In [ ]:
len(pdb_to_study_unique)

In [ ]:
len(table_proteins_PDB_CNS_membrane_l100_MemProtMD['Uniprot'].unique())

In [ ]:
table_proteins_PDB_CNS_membrane_l100_MemProtMD['PDB'].unique()

In [ ]:
pdb_to_study_unique.sort()
pdb_to_study_unique

In [ ]:
path_PDB_download = path_dir[directory5]
path_PDB_down_tocopy = path_dir[directory5A]

path_PDB_download_items = os.listdir(path_PDB_download)


for PDB_downloaded in path_PDB_download_items:
    
    split_string = PDB_downloaded.split('_')
    name = split_string[0]
    
    if name in pdb_to_study_unique:
        path_PDB_copy =path_PDB_down_tocopy+'/'+PDB_downloaded
        path_PDB = path_PDB_download + '/' +PDB_downloaded
        if not os.path.exists(path_PDB_copy):
            print(PDB_downloaded)
            shutil.copyfile(path_PDB, path_PDB_copy)


In [ ]:
zips = []
import os
for file in os.listdir(path_PDB_down_tocopy): 
    file_splitted = file.split('_')
    if len(file_splitted[0]) == 4: 
        zips.append(file)

In [ ]:
len(zips)

In [ ]:
path_PDB_down_tocopy_unzipped = path_PDB_down_tocopy+'/'+'unzipped'
pdbs_ids_unzipped = []

for file_unzipped in os.listdir(path_PDB_down_tocopy_unzipped): 
    pdb_id_unzip = file_unzipped.split('_') 
    pdbs_ids_unzipped.append(pdb_id_unzip[0])

In [ ]:
import zipfile

for file in os.listdir(path_PDB_down_tocopy):
    
    
    pdb_file_zip = file.split('_')
    pdb_id_zipped = pdb_file_zip[0]
    
    if pdb_id_zipped not in pdbs_ids_unzipped and len(pdb_id_zipped) == 4:
        print(file)
        with zipfile.ZipFile(path_dir[directory5A]+'/'+file, 'r') as zip_ref:
            zip_ref.extractall(path_dir[directory5B]+'/'+pdb_id_zipped+'_default_dppc-atomistic-simulation_extra')